In [62]:
import requests as re
import os
import pandas as pd
import numpy as np
import sqlite3
from sqlite3 import Error

four_api = os.environ['FOURSQUARE_API_KEY']
yelp_api = os.environ['YELP_API_KEY']
google_api = os.environ['GOOGLE_API_KEY']

four_header_dict = {
    'Accept': 'application/json',
    'Authorization': four_api
}

yelp_header_dict = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {yelp_api}'
}

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
    except Error as e:
        print(f"The error '{e}' occurred")

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [42]:
four_url = 'https://api.foursquare.com/v3/'
four_search = 'places/search'
places = 'bar'
radius = '1000'
loc = 'll=37.8044%2C-122.2712'
returns = 'name%2Ccategories%2Clocation%2Crating%2Cpopularity%2Cmenu'
limit = '50'
four_params = f'?query={places}&{loc}&radius={radius}&fields={returns}&limit={limit}'
four_response = re.get(four_url + four_search + four_params, headers = four_header_dict)
four_res = four_response.json()

Parse through the response to get the POI details you want (rating, name, location, etc)

In [43]:
for result in four_res['results']:
    result['location'] = result['location']['formatted_address']
    result['categories'] = result['categories'][0]['name']
four_res = four_res['results']
for result in four_res:
    if 'rating' not in result:
        result['rating'] = 11
    if 'popularity' not in result:
        result['popularity'] = 2

In [44]:
connect = sqlite3.connect('poi.db')

In [45]:
execute_query(connect, 'DROP TABLE foursquare_places')

In [46]:
create_places_four = f'''
CREATE TABLE IF NOT EXISTS foursquare_places (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    category TEXT,
    location TEXT,
    popularity DECIMAL,
    rating DECIMAL
);
'''

execute_query(connect, create_places_four)

In [47]:
for result in four_res:
    insert_place = f"""
    INSERT INTO foursquare_places (name, category, location, popularity, rating)
    VALUES ("{result['name']}", "{result['categories']}", "{result['location']}", {result['popularity']}, {result['rating']});
    """
    execute_query(connect, insert_place)

Put your parsed results into a DataFrame

In [48]:
four_data = pd.read_sql_query('SELECT * FROM foursquare_places', connect, index_col='id')

In [49]:
four_data.head()

,name,category,location,popularity,rating
id,,,,,
1,Bar Shiru,Cocktail Bar,"1611 Telegraph Ave, Oakland, CA 94612",0.974027,8.9
2,Bar 355,Bar,"355 19th St, Oakland, CA 94612",0.923766,7.9
3,Twelve Bar,Bar,"1001 Broadway (11th Street), Oakland, CA 94607",0.867315,6.6
4,Stinky’s Bar,Bar,"1644 Telegraph Ave (17th), Oakland, CA 94612",0.944793,11.0
5,Cafe Van Kleef,Art Gallery,"1621 Telegraph Ave (at 17th St), Oakland, CA 9...",0.979221,8.9


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [50]:
yelp_url = 'https://api.yelp.com/v3/businesses/'
yelp_search = 'search'
lat = '37.8044'
long = '-122.2712'
yelp_params = f'?term={places}&latitude={lat}&longitude={long}&radius={radius}&limit={limit}'
yelp_response = re.get(yelp_url + yelp_search + yelp_params, headers = yelp_header_dict)
yelp_res = yelp_response.json()

Parse through your result and get POI details

In [51]:
yelp_info = []
for business in yelp_res['businesses']:
    bid_id = business['id']
    bid_info = re.get(yelp_url + bid_id, headers = yelp_header_dict).json()
    bid = {}
    bid['name'] = business['name']
    bid['categories'] = business['categories'][0]['title']
    bid['location'] = ' '.join(business['location']['display_address'])
    bid['rating'] = bid_info['rating']
    bid['popularity'] = business['review_count']
    yelp_info.append(bid)

Put your parsed results into a DataFrame

In [52]:
execute_query(connect, 'DROP TABLE yelp_places')

In [53]:
create_places_yelp = f'''
CREATE TABLE IF NOT EXISTS yelp_places (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    category TEXT,
    location TEXT,
    popularity INTEGER,
    rating DECIMAL
);
'''

execute_query(connect, create_places_yelp)

In [54]:
for result in yelp_info:
    insert_place = f"""
    INSERT INTO yelp_places (name, category, location, popularity, rating)
    VALUES ("{result['name']}", "{result['categories']}", "{result['location']}", {result['popularity']}, {result['rating']});
    """
    execute_query(connect, insert_place)

In [55]:
yelp_data = pd.read_sql_query('SELECT * FROM yelp_places', connect, index_col='id')

In [56]:
yelp_data.head()

,name,category,location,popularity,rating
id,,,,,
1,Night Heron,Bars,"1780 Telegraph Ave Oakland, CA 94612",5,5.0
2,Hello Stranger,Bars,"1724 Broadway Oakland, CA 94612",156,4.0
3,Mad Oak,Bars,"135 12th St Oakland, CA 94607",713,4.0
4,First Edition,Cocktail Bars,"1915 Broadway Oakland, CA 94612",72,4.0
5,The Miranda,Cocktail Bars,"1739 Broadway Oakland, CA 94612",110,4.0


# Google (stretch)

In [85]:
google_search = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
g_loc = '37.8044%2C-122.2712'
google_params = f'keyword={places}&location={g_loc}&radius={radius}&limit={limit}&key={google_api}'
google_response = re.get(google_search + google_params)
google_json = google_response.json()

In [86]:
google_info = []
for result in google_json['results']:
    bid_id = result['place_id']
    bid_info = re.get(f'https://maps.googleapis.com/maps/api/place/details/json?fields={g_fields}&place_id={bid_id}&key={google_api}').json()
    bid = {}
    bid['name'] = result['name']
    bid['rating'] = result['rating']
    bid['categories'] = result['types'][0]
    bid['popularity'] = bid_info['result']['user_ratings_total']
    bid['location'] = bid_info['result']['formatted_address']
    google_info.append(bid)

In [ ]:
execute_query(connect, 'DROP TABLE google_places')

In [88]:
create_places_google = f'''
CREATE TABLE IF NOT EXISTS google_places (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    category TEXT,
    location TEXT,
    popularity INTEGER,
    rating DECIMAL
);
'''

execute_query(connect, create_places_google)

In [89]:
for result in google_info:
    insert_place = f"""
    INSERT INTO google_places (name, category, location, popularity, rating)
    VALUES ("{result['name']}", "{result['categories']}", "{result['location']}", {result['popularity']}, {result['rating']});
    """
    execute_query(connect, insert_place)

In [90]:
google_data = pd.read_sql_query('SELECT * FROM google_places', connect, index_col='id')

In [91]:
google_data.head()

,name,category,location,popularity,rating
id,,,,,
1,Hello Stranger,bar,"1724 Broadway, Oakland, CA 94612, USA",337,4.3
2,Bar 355,bar,"355 19th St, Oakland, CA 94612, USA",211,4.5
3,Merchants' Saloon,bar,"Shasta Produce, 401 2nd St, Oakland, CA 94607,...",429,4.5
4,Somar Bar and Lounge,bar,"1727 Telegraph Ave, Oakland, CA 94612, USA",416,4.4
5,DREXL,bar,"382 19th St, Oakland, CA 94612, USA",277,4.5


Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Get the top 10 restaurants according to their rating

In [92]:
yelp10 = yelp_data.sort_values('rating', ascending = False).head(10)
foursquare10 = four_data[four_data['rating'] <= 10].sort_values('rating', ascending = False).head(10)
google10 = google_data.sort_values('rating', ascending = False).head(10)

In [60]:
yelp10

,name,category,location,popularity,rating
id,,,,,
1,Night Heron,Bars,"1780 Telegraph Ave Oakland, CA 94612",5,5.0
28,Binny’s,Cocktail Bars,"532 8th St Oakland, CA 94607",42,5.0
47,Two Pitchers Brewing,Beer Gardens,"2344 Webster St Oakland, CA 94612",50,5.0
40,Dinner House,Beer Bar,"329 14th St Oakland, CA 94612",23,5.0
32,Tamarack,Cocktail Bars,"1501 Harrison St Oakland, CA 94612",10,5.0
29,The Kon Tiki Room,Tiki Bars,"1900 Telegraph Ave Oakland, CA 94612",5,5.0
26,Sante Adairius Oakland Arbor,Brewpubs,"460 8th St Oakland, CA 94607",5,5.0
20,The Gallery Art Bar,Art Galleries,"310 14th St Oakland, CA 94612",11,4.5
10,Viridian,Bars,"2216 Broadway Oakland, CA 94612",235,4.5


In [61]:
foursquare10

,name,category,location,popularity,rating
id,,,,,
46,Drake's Dealership,Beer Garden,"2325 Broadway (at 23rd St), Oakland, CA 94612",0.991932,9.3
9,Mad Oak,Cocktail Bar,"135 12th St (at Madison St), Oakland, CA 94607",0.991213,9.2
20,Telegraph Bar and Beer Garden,Beer Garden,"2318 Telegraph Ave (at 23rd St), Oakland, CA 9...",0.975767,9.0
1,Bar Shiru,Cocktail Bar,"1611 Telegraph Ave, Oakland, CA 94612",0.974027,8.9
5,Cafe Van Kleef,Art Gallery,"1621 Telegraph Ave (at 17th St), Oakland, CA 9...",0.979221,8.9
7,Drexl,Beer Bar,"382 19th St (btwn Webster & Franklin), Oakland...",0.959189,8.9
10,Viridian,Cocktail Bar,"2216 Broadway (btw Grand & 22nd), Oakland, CA ...",0.979166,8.9
17,The Port Bar,Restaurant,"2023 Broadway, Oakland, CA 94612",0.982648,8.5
44,The Punchdown,Wine Bar,"1737 Broadway, Oakland, CA 94612",0.968832,8.3


In [93]:
google10

,name,category,location,popularity,rating
id,,,,,
17,Binny's,bar,"532 8th St, Oakland, CA 94607, USA",36,4.9
11,Low Bar,bar,"2300 Webster St, Oakland, CA 94612, USA",92,4.8
13,Bar Shiru,bar,"1611 Telegraph Ave #100, Oakland, CA 94612, USA",210,4.8
10,Penelope,bar,"555 12th St Ste. 120, Oakland, CA 94607, USA",51,4.6
9,First Edition,bar,"1915 Broadway, Oakland, CA 94612, USA",106,4.5
19,The Kon-Tiki,bar,"347 14th St #3906, Oakland, CA 94612, USA",482,4.5
12,Cafe Van Kleef,bar,"1621 Telegraph Ave, Oakland, CA 94612, USA",895,4.5
2,Bar 355,bar,"355 19th St, Oakland, CA 94612, USA",211,4.5
20,The Hatch Oakland,bar,"402 15th St, Oakland, CA 94612, USA",264,4.5


# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).